In [ ]:

!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@575ef4d

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
* [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

### Training from scratch (LoRA).

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "<hf_token>", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Training from scratch for LoRA! We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Training from pretrained LoRA

In [ ]:
from unsloth import FastLanguageModel
import torch

Load pretrained from local weights (from a folder called lora_model)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "unsloth/llama-3.2-3b-instruct-bnb-4bit", # MODEL
        max_seq_length = 2048,
        dtype = None,
        load_in_4bit = True,
    )

Load pretrained from uploaded weights

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "<hf_username>/<hf_repo_name>",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token = "<hf_token>",
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: NVIDIA A10. Max memory: 21.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [4]:
from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.1",
# )

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

# A dataset that contains a column called "conversations"
# Each data point is addressed by dataset[index]
# To get conversations: dataset[index]["conversations"]
# dataset[index]["conversations"] gives a list of 2 dictionaries. First dict is user, second gpt


We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

```
from datasets import Dataset
pdata = {"conversations":[[{"from": "system", "value": "You are an expert in correcting typos in sentences."}, {"from": "human", "value": "You are an assistant"}, {"from": "gpt", "value": "yes"}], [{"from": "human", "value": "hello"}, {"from": "gpt", "value": "hi there"}]],
         "scource":["None", "None"],
         "score": [0, 0]}
play_dataset = Dataset.from_dict(pdata)
```

In [ ]:
from datasets import load_from_disk, Dataset
dataset = load_from_disk("phone_ds") # change to zoom_ds for the zoom dataset
print(dataset)
print(dataset[0])

Dataset({
    features: ['conversations'],
    num_rows: 36924
})
{'conversations': [{'from': 'system', 'value': 'You are an expert in correcting typos in sentences.'}, {'from': 'human', 'value': '\nHere are examples of sentences with typos; learn from them:\n\n    sentence: bh 1489 genetic engin555s 6ikk havexc558tedtokg8ni3m3 caj8hk5 if su5vivinh in so8ce 3ithout kif6 suiji5t\n    corrected: by 2480 genetic engineers will have created organisms capable of surviving in space without life support\n\n    sentence: bh 1510 th5 kocak 6ove5nm6nt 3ikk have been inv5sting in g5een inftakt58ct8f6 fo5 g6n558tiink\n    corrected: by 2510 the local government will have been investing in green infrastructure for generations\nNow, please correct this sentence and output only the corrected version with no additional text:\n\ntomo55o3 8ft65noon ikk he fivisinh the ove5h5o3n 9e5enni8ks\n        '}, {'from': 'gpt', 'value': 'tomorrow afternoon ill be dividing the overgrown perennials'}]}


In [6]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)
# testset = standardize_sharegpt(testset)
# testset = testset.map(formatting_prompts_func, batched = True,)

We look at how the conversations are structured for item 5:

In [ ]:
dataset[1]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [ ]:
dataset[1]["text"]

In [ ]:
dataset[1]["conversations"]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Applying chat template to train dataset (num_proc=2): 100%|██████████| 36924/36924 [00:04<00:00, 8975.89 examples/s] 
Truncating train dataset (num_proc=2): 100%|██████████| 36924/36924 [00:06<00:00, 5876.38 examples/s]


We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [8]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map: 100%|██████████| 36924/36924 [00:06<00:00, 5830.79 examples/s]


We verify masking is actually done:

In [9]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 02 Mar 2025\n\nYou are an expert in correcting typos in sentences.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHere are examples of sentences with typos; learn from them:\n\n    sentence: hy 248u g5netic enh8neers 3ikk hav5 created organisms ca8ableaof survivin6 in siac5 3ithour life suiiort\n    corrected: by 2480 genetic engineers will have created organisms capable of surviving in space without life support\n\n    sentence: by 151k the kocak government sikk have7been 8nvestinh in green infrastructure for henerarions\n    corrected: by 2510 the local government will have been investing in green infrastructure for generations\nNow, please correct this sentence and output only the corrected version with no additional text:\n\n5esen5che5s hnve been stusying 3nada9tame5s short dna se1hencesvwirh siecific bindingy9roie5ties for variohs ni9kications<|

In [10]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                               \n\nresearchers have been studying dna aptamers short dna sequences with specific binding properties for various applications<|eot_id|>'

We can see the System and Instruction prompts are successfully masked!

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10. Max memory = 21.988 GB.
2.295 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 36,924 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 4,615
 "-____-"     Number of trainable parameters = 24,313,856


In [13]:
model.save_pretrained("phone_model")  # Local saving
tokenizer.save_pretrained("phone_model")

('phone_model/tokenizer_config.json',
 'phone_model/special_tokens_map.json',
 'phone_model/tokenizer.json')

In [ ]:
model.push_to_hub("<hf_username>/<hf_repo_name>", token = "<hf_token>") # Online saving
tokenizer.push_to_hub("<hf_username>/<hf_repo_name>", token = "<hf_token>") # Online saving